#  PyTorch Lightning

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://lightning.ai/docs/pytorch/stable/starter/introduction.html
* https://lightning.ai/docs/pytorch/stable/levels/core_skills.html
* https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.core.LightningModule.html#lightning.pytorch.core.LightningModule.log
* https://lightning.ai/docs/pytorch/stable/extensions/logging.html
* https://lightning.ai/docs/pytorch/stable/common/progress_bar.html
* https://lightning.ai/docs/pytorch/stable/common/early_stopping.html
* https://lightning.ai/docs/pytorch/1.6.3/api/pytorch_lightning.utilities.model_summary.html#pytorch_lightning.utilities.model_summary.ModelSummary
* https://torchmetrics.readthedocs.io/en/stable/pages/lightning.html
* https://pytorch-lightning.readthedocs.io/en/2.1.2/pytorch/
* https://www.youtube.com/watch?v=XbIN9LaQycQ&list=PLhhyoLH6IjfyL740PTuXef4TstxAK6nGP
* https://pytorch-lightning.readthedocs.io/en/2.1.2/pytorch/data/datamodule.html

## Задачи для совместного разбора

1\. Создайте датасет для классификации и обучите модель при помощи PyTorch Lightning.

In [ ]:
!pip install pytorch_lightning

In [7]:
!pip install torchmetrics

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from typing import Any, List, Optional, Union
import pytorch_lightning as pl

class MyLightningModule(pl.LightningModule):
    """
    Класс модуля PyTorch Lightning.

    Этот класс определяет структуру модели, шаги обучения, валидации и тестирования,
    а также настройки оптимизатора для обучения.
    """

    def __init__(self):
        """Здесь определяется архитектура модели и инициализируются все необходимые слои."""
        super().__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Определяет прямой проход модели.

        Args:
            x (torch.Tensor): Входной тензор.

        Returns:
            torch.Tensor: Выходной тензор модели.
        """

    def training_step(self, batch: Any, batch_idx: int) -> torch.Tensor:
        """
        Выполняет один шаг обучения.

        Args:
            batch (Any): Батч данных для обучения.
            batch_idx (int): Индекс текущего батча.

        Returns:
            torch.Tensor: Значение функции потерь для этого шага.
        """

    def validation_step(self, batch: Any, batch_idx: int) -> None:
        """
        Выполняет один шаг валидации.

        Args:
            batch (Any): Батч данных для валидации.
            batch_idx (int): Индекс текущего батча.
        """

    def test_step(self, batch: Any, batch_idx: int) -> None:
        """
        Выполняет один шаг тестирования.

        Args:
            batch (Any): Батч данных для тестирования.
            batch_idx (int): Индекс текущего батча.
        """

    def configure_optimizers(self) -> torch.optim.Optimizer:
        """
        Настраивает оптимизатор для обучения модели.

        Returns:
            torch.optim.Optimizer: Настроенный оптимизатор.
        """

In [9]:
import torch  as th
import torch.nn
from torch.utils.data import TensorDataset
from torchmetrics import Accuracy

In [6]:
num_samples, num_features = 1000, 10

train_size = int(num_samples * 0.8)
X = th.randn(num_samples, num_features)
y = th.randint(0, 2, (num_samples, ))

dataset_train = TensorDataset(X[:train_size], y[:train_size])
dataset_val = TensorDataset(X[train_size:], y[train_size:])

In [25]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from typing import Any, List, Optional, Union
import pytorch_lightning as pl

class MyLightningModule(pl.LightningModule):
    """
    Класс модуля PyTorch Lightning.

    Этот класс определяет структуру модели, шаги обучения, валидации и тестирования,
    а также настройки оптимизатора для обучения.
    """

    def __init__(self, n_inputs: int, n_hidden: int, n_out: int) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_inputs, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_out),
        )

        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="binary")


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


    def training_step(self, batch: Any, batch_idx: int) -> torch.Tensor:
        x, y = batch
        logits = self(x) # self.forward(x)
        loss = self.criterion(logits, y)

        # доп. метрики
        preds = logits.argmax(dim=1)
        acc = self.accuracy(preds, y)

        # logging
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)

        return loss

    def validation_step(self, batch: Any, batch_idx: int) -> None:
        x, y = batch
        logits = self(x) # self.forward(x)

        # доп. метрики
        preds = logits.argmax(dim=1)
        acc = self.accuracy(preds, y)
        self.log("val_acc", acc, prog_bar=True)


    def test_step(self, batch: Any, batch_idx: int) -> None:
        x, y = batch
        logits = self(x) # self.forward(x)

        # доп. метрики
        preds = logits.argmax(dim=1)
        acc = self.accuracy(preds, y)
        self.log("test_acc", acc, prog_bar=True)

    def configure_optimizers(self) -> torch.optim.Optimizer:
        return th.optim.Adam(self.parameters(), lr=0.001)

In [23]:
from torch.utils.data import DataLoader

class MyDataModule(pl.LightningDataModule):
    def __init__(self, num_samples: int, num_features: int, batch_size: int):
        super().__init__()
        self.num_samples = num_samples
        self.num_features = num_features
        self.batch_size = batch_size

    def setup(self, stage: Optional[str] = None) -> None:
        """
        Настраивает данные для использования на каждом этапе (обучение/валидация/тестирование).

        Args:
            stage (Optional[str]): Этап, на котором вызывается метод ("fit" или "test").
        """
        train_size = int(self.num_samples * 0.8)
        X = th.randn(self.num_samples, self.num_features)
        y = th.randint(0, 2, (self.num_samples, ))

        self.dataset_train = TensorDataset(X[:train_size], y[:train_size])
        self.dataset_val = TensorDataset(X[train_size:], y[train_size:])

    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self) -> DataLoader:
        return DataLoader(self.dataset_val, batch_size=self.batch_size, shuffle=False)

In [27]:
model = MyLightningModule(10, 5, 2)
data_module = MyDataModule(1000, 10, 32)

trainer = pl.Trainer(
    max_epochs=10,
    log_every_n_steps=1,
)
trainer.fit(model, data_module)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Sequential       | 67     | train
1 | criterion | CrossEntropyLoss | 0      | train
2 | accuracy  | BinaryAccuracy   | 0      | train
-------------------------------------------------------
67        Trainable params
0         Non-trainable params
67        Total params
0.000     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
class MyDataModule(pl.LightningDataModule):
    """
    Класс модуля данных PyTorch Lightning.

    Этот класс отвечает за загрузку, подготовку и предоставление данных
    для обучения, валидации и тестирования модели.
    """

    def __init__(self):
        """
        Инициализирует модуль данных.

        Args:
            data_dir (str): Путь к директории с данными.
        """
        super().__init__()

    def prepare_data(self) -> None:
        """
        Подготавливает данные для использования.

        Здесь можно выполнить загрузку данных или другие подготовительные операции.
        """
        pass

    def setup(self, stage: Optional[str] = None) -> None:
        """
        Настраивает данные для использования на каждом этапе (обучение/валидация/тестирование).

        Args:
            stage (Optional[str]): Этап, на котором вызывается метод ("fit" или "test").
        """

    def train_dataloader(self) -> DataLoader:
        """
        Возвращает DataLoader для обучающих данных.

        Returns:
            DataLoader: DataLoader с обучающими данными.
        """

    def val_dataloader(self) -> DataLoader:
        """
        Возвращает DataLoader для данных валидации.

        Returns:
            DataLoader: DataLoader с данными валидации.
        """


    def test_dataloader(self) -> DataLoader:
        """
        Возвращает DataLoader для тестовых данных.

        Returns:
            DataLoader: DataLoader с тестовыми данными.
        """

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Загрузите набор данных из файла `Walmart.csv`. Выполните следующую процедуру предобработки:
- замените цены `Weekly_Sales` на логарифм цены;
- удалите столбец с датами;
- закодируйте столбцы `Store` и `Holiday_Flag` при помощи `TargetEncoder` (см. пакет [category_encoders](https://contrib.scikit-learn.org/category_encoders/));
- после кодирование выполните стандартизацию признаков;
- разбейте выборку на обучающее, валидационное и тестовое множество.

Все преобразования допускает делать при помощи `numpy`, `pandas` и `sklearn`.

- [ ] Проверено на семинаре

<p class="task" id="2"></p>

2\. В ячейках ниже представлен шаблонный код для обучения модели. В данной версии все реализовано "с нуля": обучение, метрики, визуализация, логирование, логика ранней остановки.

Используя набор данных из предыдущего задания, обучите модель, используя предложенную реализацию. Визуализируйте динамику изменения среднего значения функции потерь и метрик на обучающем и валидационном множестве. Интегрируйте реализацию ранней остановки в цикл обучения. Посчитайте и выведите на экран значения метрик на тестовом множестве.

- [ ] Проверено на семинаре

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def r2_score(y_true, y_pred):
    total_sum_squares = torch.sum((y_true - y_true.mean())**2)
    residual_sum_squares = torch.sum((y_true - y_pred)**2)
    r2 = 1 - (residual_sum_squares / total_sum_squares)
    return r2

def mape_score(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, n_inputs, h_hidden):
        super().__init__()
        self.fc1 = nn.Linear(n_inputs, h_hidden)
        self.fc2 = nn.Linear(h_hidden, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = out.relu()
        out = self.fc2(out)
        return out

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, early_stopping):
    train_losses = []
    val_losses = []
    train_r2s = []
    val_r2s = []
    train_mapes = []
    val_mapes = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_r2 = 0.0
        train_mape = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).flatten()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_r2 += r2_score(targets, outputs).item()
            train_mape += mape_score(targets, outputs).item()

        train_loss /= len(train_loader)
        train_r2 /= len(train_loader)
        train_mape /= len(train_loader)

        train_losses.append(train_loss)
        train_r2s.append(train_r2)
        train_mapes.append(train_mape)

        model.eval()
        val_loss = 0.0
        val_r2 = 0.0
        val_mape = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs).flatten()
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                val_r2 += r2_score(targets, outputs).item()
                val_mape += mape_score(targets, outputs).item()

        val_loss /= len(val_loader)
        val_r2 /= len(val_loader)
        val_mape /= len(val_loader)

        val_losses.append(val_loss)
        val_r2s.append(val_r2)
        val_mapes.append(val_mape)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # здесь должна быть логика ранней остановки

    return train_losses, val_losses, train_r2s, val_r2s, train_mapes, val_mapes

In [ ]:
batch_size = 32
learning_rate = 0.01
patience = 5
num_epochs = 100

<p class="task" id="3"></p>

3\. Перепишите логику обучения модели, используя `pytorch_lightning`. Для расчета метрик $R^2$ и MAPE используйте `torchmetrics`. Ранняя остановка в данном задании не требуется. После завершения обучения посчитайте значения метрик на тестовом множестве.

В процессе обучения настройки progressbar так, что:
* для каждого батча во время обучения рассчитывается значение функции потерь и метрик, по завершению эпохи показатели усредняются;
* для каждого батча во время валидации рассчитывается значение функции потерь и метрик, по завершению эпохи показатели усредняются.

- [ ] Проверено на семинаре

<p class="task" id="4"></p>

4\. Повторите задачу 3, добавив логику ранней остановки, используя callback `pytorch_lightning`. Если значение функции потерь на валидационном множестве не улучшалось в течении 5 эпох, происходит ранняя остановка.

- [ ] Проверено на семинаре

<p class="task" id="5"></p>

5\. Повторите задачу 4, оформив набор данных в виде `pytorch_lightning.LightningDataModule`. Всю логику по созданию датасета (преобразования признаков, разбиение и т.д.) запакуйте в метод `setup`.

- [ ] Проверено на семинаре

<p class="task" id="6"></p>

6\. Повторите задачу 5, добавив логирование при помощи `wandb`.

Вставьте в текстовую ячейку скриншоты, демонстрирующие интерфейс `wandb` со всеми нужными визуализациями.

- [ ] Проверено на семинаре